# Cherry Pick script with PyLabRobot for Hamilton

# questions 

Does lld work? 

# To do

- [X] Copy Hamilton 223 specs from lab to fit in PLR <br>
- [ ] Test pipeting in each position to ensure the sites and carriers are correctly located
- [ ] Write code for cherry pick <br>
- [ ] Simulate code on PLR simulator <br>
- [ ] Test code with water.  <br>
- [ ] Test with foil seal. Can i get it to override LLD? <br>
- [ ] Implement in cherry_pick git repo <br>
- [ ] Test if i can pearse the seals <br>
- [ ] Test if i can split into the three parts. <br>


# Fun to do

Test using the ISWAP to move plates or remove lids <br>


In [1]:
# await lh.stop()
# await vis.stop()

In [2]:
# !pip install pyusb

# Settup Simulator or Backend

In [3]:
simulate = False
# simulate = True


from pylabrobot.liquid_handling import LiquidHandler


from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer

# if simulate:
#   # from pylabrobot.liquid_handling.backends.simulation.simulator_backend import SimulatorBackend
#   # from pylabrobot.liquid_handling import LiquidHandler
#   # from pylabrobot.liquid_handling.backends import ChatterBoxBackend
#   # from pylabrobot.visualizer.visualizer import Visualizer
# else:
#   from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.liquid_handling.backends import STAR

# from pylabrobot.resources.hamilton import HamiltonSTARDeck
from pylabrobot.resources.hamilton import STARDeck
# from pylabrobot.resources.hamilton import STARLetDeck

STAR_SIZE_X=1900
STAR_SIZE_Y=653.5
STAR_SIZE_Z=900
if simulate:
  # sb = SimulatorBackend(open_browser=False)
  # lh = LiquidHandler(backend=sb, deck=HamiltonSTARDeck(num_rails=55, size_x=STAR_SIZE_X, size_y=STAR_SIZE_Y, size_z=STAR_SIZE_Z))
  lh = LiquidHandler(backend=ChatterBoxBackend(), deck=STARDeck())
  vis = Visualizer(resource=lh)
else:
  backend = STAR()
  lh = LiquidHandler(backend=backend, deck=STARDeck())

# vis = Visualizer(resource=lh)
# await vis.setup()
await lh.setup()

if simulate:
  await vis.setup()


# if simulate:
#   sb.wait_for_connection()


# Work table sesources
from pylabrobot.resources import (
  TIP_CAR_480_A00,
    # PLT_CAR_L5AC_A00,
    # Cos_96_DW_1mL,
    # HTF_L
  TIP_CAR_NTR_A00,
  # MFXCarrier, # Think is: MFX_CAR_3TipBC_2NTR1
  # ShakerCarrier, # think is: CVS_Shaker0_A00
  PLT_CAR_L5AC,
)


# labware resources:
from pylabrobot.resources import (
  TIP_50ul_L,
  STF_L, # standard volume 300ul tips with filter
  corning_96_wellplate_360ul_flat, # OT plate type
  Cos_96_FL # corning alternative to OT plate
)


from pylabrobot.resources.carrier import MFXCarrier, create_homogeneous_carrier_sites, Coordinate

## there are no tube runners in PLR yet.


from pylabrobot.resources.trash import Trash

trash = lh.deck.get_trash_area()


# Settup Deck

In [4]:
# dummy CSV_Shaker0


from pylabrobot.resources.carrier import ShakerCarrier, create_homogeneous_carrier_sites, Coordinate
# Define the carrier name and dimensions
name = "my_shaker_carrier"

size_x=157.5
size_y=497.0
size_z=90.0
site_locations1=[
    # Coordinate(1.1, 167.5, 25.5), # Site.1.(X,Y,Z) !!!
    Coordinate(14.6, 172.1, 77), # Site.2.(X,Y,Z)
    # Coordinate(1.2, 48.5, 25.5), # Site.3.(X,Y,Z) !!!
    Coordinate(14.6, 53, 77),    # Site.4.(X,Y,Z)
    Coordinate(14.6, 53.0, 20.5),# Site.5.(X,Y,Z)
    Coordinate(14.6, 392.1, 75), # Site.6.(X,Y,Z)
    Coordinate(14.6, 286.9, 75)  # Site.7.(X,Y,Z)
  ]
site_size_x1=127
site_size_y1=86

# site_locations2=[
#     Coordinate(1.1, 167.5, 25.5), # Site.1.(X,Y,Z)
#     Coordinate(1.2, 48.5, 25.5), # Site.3.(X,Y,Z)
#   ]
# site_size_x2=154
# site_size_y2=95

sites1 = create_homogeneous_carrier_sites(site_locations1, site_size_x1, site_size_y1)
# sites2 = create_homogeneous_carrier_sites(site_locations2, site_size_x2, site_size_y2)

# Create an instance of the ShakerCarrier class
shaker_carrier = ShakerCarrier(name, size_x, size_y, size_z, sites1)

location = Coordinate(x=77.5, y=63.0, z=100.0)
# lh.deck.assign_child_resource(shaker_carrier, rails=1, reassign=True)
lh.deck.assign_child_resource(shaker_carrier, rails=None, location=location, reassign=True)



In [5]:
# Work table sesources
# from pylabrobot.resources import (
#   TIP_CAR_NTR_A00,
# )

## Tip positioning
# tip_car = TIP_CAR_480_A00(name='tip_carrier') ### OPS OPS USING THE BIG TIP CARRIER!!!!

# tip_car = TIP_CAR_NTR_A00(name='tip_carrier')

from pylabrobot.resources.carrier import TipCarrier, create_homogeneous_carrier_sites

# attempt to build own tip tip settup original z-value = 29.0
def TIP_CAR_NTR_A00(name: str) -> TipCarrier:
  """ Carrier with 5 nestable tip rack positions """
  return TipCarrier(
    name=name,
    size_x=135.0,
    size_y=497.0,
    size_z=130.0, # 200.0 # 130
    sites=create_homogeneous_carrier_sites([
        Coordinate(6.2, 10.0, 63.4),
        Coordinate(6.2, 106.0, 63.4),
        Coordinate(6.2, 202.0, 63.4),
        Coordinate(6.2, 298.0, 63.4),
        Coordinate(6.2, 394.0, 63.4)
      ],
      site_size_x=122.4,
      site_size_y=82.6,
    ),
    model="TIP_CAR_NTR_A00"
  )


tip_car = TIP_CAR_NTR_A00(name='tip_carrier')

In [6]:
from pylabrobot.resources.carrier import TubeCarrier, PlateCarrier, create_homogeneous_carrier_sites, Coordinate

plt_car1 = PLT_CAR_L5AC(name='PLT_CAR_L5AC_1')
# # plt_car[3] = corning_96_wellplate_360ul_flat(name='plt01')
# # plt_car[4] = corning_96_wellplate_360ul_flat(name='plt02')
# plt_car[3] = Cos_96_FL(name='plt01')
# plt_car[4] = Cos_96_FL(name='plt02')

plt_car2 = PLT_CAR_L5AC(name='PLT_CAR_L5AC_2')


# MFXCarrier

# Define the carrier name and dimensions
name = "my_mfx_carrier"
size_x = 135.0 # Dim.Dx
size_y = 497.0 # Dim.Dy
size_z = 130.0 # Dim.Dz

# Define the carrier sites
site_locations = [
                  Coordinate(x=6.3, y=202.2, z=114.7), # Site.1.(X,Y,Z)
                  Coordinate(x=6.3, y=106.2, z=77.0),  # Site.2.(X,Y,Z)
                  Coordinate(x=6.3, y=10.2, z=77.0),   # Site.3.(X,Y,Z)
                  Coordinate(x=6.3, y=394.2, z=114.7), # Site.4.(X,Y,Z)
                  Coordinate(x=6.3, y=298.2, z=114.7), # Site.5.(X,Y,Z)
                  ]
site_size_x = 122.4 # Site.1.Dx122.4
site_size_y = 82.6  # Site.1.Dy82.6
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)

mfx_carrier = MFXCarrier(name, size_x, size_y, size_z, sites)


# Reagent reservar 1:

name = "my_RGT_CAR_3R_A01_1"
size_x = 22.5 # Dim.Dx
size_y = 497 # Dim.Dy
size_z = 136.0 # Dim.Dz

# Define the carrier sites
site_locations = [
                  Coordinate(x=0.75, y=9.1, z=18.5), # Site.1.(X,Y,Z)
                  Coordinate(x=0.75, y=335.9, z=18.5),  # Site.2.(X,Y,Z)
                  Coordinate(x=0.55, y=172.5, z=18.5),   # Site.3.(X,Y,Z)
                  ]
site_size_x = 21 # Site.1.Dx122.4
site_size_y = 142  # Site.1.Dy82.6
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)
reservar_carrier1 = TubeCarrier(name, size_x, size_y, size_z, sites)


name = "my_RGT_CAR_3R_A01_2"
size_x = 22.5 # Dim.Dx
size_y = 497 # Dim.Dy
size_z = 136.0 # Dim.Dz

# Define the carrier sites
site_locations = [
                  Coordinate(x=0.75, y=9.1, z=18.5), # Site.1.(X,Y,Z)
                  Coordinate(x=0.75, y=335.9, z=18.5),  # Site.2.(X,Y,Z)
                  Coordinate(x=0.55, y=172.5, z=18.5),   # Site.3.(X,Y,Z)
                  ]
site_size_x = 21 # Site.1.Dx122.4
site_size_y = 142  # Site.1.Dy82.6
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)
reservar_carrier2 = TubeCarrier(name, size_x, size_y, size_z, sites)


# Reagent reservar 2:
name = "my_RGT_CAR_5R_A00"
size_x = 22.5 # Dim.Dx
size_y = 497 # Dim.Dy
size_z = 82.0 # Dim.Dz

# Define the carrier sites
site_locations = [
                  Coordinate(x=1.25, y=198.5, z=18.5), # Site.1.(X,Y,Z)
                  Coordinate(x=1.25, y=102.5, z=18.5),  # Site.2.(X,Y,Z)
                  Coordinate(x=1.25, y=6.5, z=18.5),   # Site.3.(X,Y,Z)
                  Coordinate(x=1.25, y=390.5, z=18.5),   # Site.4.(X,Y,Z)
                  Coordinate(x=1.25, y=294.5, z=18.5),   # Site.5.(X,Y,Z)
                  ]
site_size_x = 20 # Site.1.Dx122.4
site_size_y = 90  # Site.1.Dy82.6
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)

reservar_carrier3 = TubeCarrier(name, size_x, size_y, size_z, sites)


# Coolers

name1 = "my_cooler_carrier_1"

size_x = 135 # Dim.Dx
size_y = 497 # Dim.Dy
size_z = 130 # Dim.Dz

# Define the carrier sites
site_locations = [
                  Coordinate(x=4, y=104.5, z=81.5), # Site.1.(X,Y,Z)
                  Coordinate(x=4, y=8.5, z=81.5),  # Site.2.(X,Y,Z)
                  Coordinate(x=4, y=392.5, z=81.5),   # Site.3.(X,Y,Z)
                  Coordinate(x=4, y=296.5, z=81.5), # Site.4.(X,Y,Z)
                  Coordinate(x=4, y=200.5, z=81.5), # Site.5.(X,Y,Z)
                  ]
site_size_x = 127 # Site.1.Dx
site_size_y = 86  # Site.1.Dy
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)

# Create an instance of the PlateCarrier
cooler_carrier1 = PlateCarrier(name1, size_x, size_y, size_z, sites)


name2 = "my_cooler_carrier_2"
sites = create_homogeneous_carrier_sites(site_locations, site_size_x, site_size_y)
cooler_carrier2 = PlateCarrier(name2, size_x, size_y, size_z, sites)


In [7]:
# Reagent reservar 3:
name = "my_SMP_CAR_24_A00"
# Dim.Dx22.5Dim.Dy497Dim.Dz140

# Settup Labware

In [8]:
from pylabrobot.resources import (
  # TIP_50ul_L,
  STF_L, # standard volume 300ul tips with filter
  # TIP_50ul_w_filter_L, # 50ul with filter landscape position
)
from pylabrobot.resources.itemized_resource import create_equally_spaced
from pylabrobot.resources.tip_rack import TipRack, TipSpot
from pylabrobot.resources.ml_star.tip_creators import (
  low_volume_tip_no_filter,
  low_volume_tip_with_filter,
  standard_volume_tip_no_filter,
  standard_volume_tip_with_filter,
  high_volume_tip_no_filter,
  high_volume_tip_with_filter,
  four_ml_tip_with_filter,
  five_ml_tip,
  fifty_ul_tip_with_filter,
  fifty_ul_tip_no_filter
)

#: Rack with 96 50ul Tip with filter
def TIP_50ul_w_filter_L(name: str, with_tips: bool = True) -> TipRack:
  return TipRack(
    name=name,
    size_x=122.4,
    size_y=82.6,
    size_z=65.0, # 18.0
    model="TIP_50ul_w_filter",
    items=create_equally_spaced(TipSpot,
      num_items_x=12,
      num_items_y=8,
      dx=7.2, # 7.2
      dy=5.3,
      dz=-92.0, # 65.5
      item_dx=9.0,
      item_dy=9.0,
      size_x=9.0,
      size_y=9.0,
      make_tip=fifty_ul_tip_with_filter,
    ),
    with_tips=with_tips
  )

# tip_car[1] = TIP_50ul_L(name='tip01')
# tip_car[1] = TIP_50ul_w_filter_L(name='tips_01_50ul') # for TIP_CAR_480_A00
# tip_car[2] = TIP_50ul_w_filter_L(name='tips_02_50ul') # for TIP_CAR_480_A00
# tip_car[0] = STF_L(name='tips_01_300ul') # for TIP_CAR_480_A00
# tip_car[3] = STF_L(name='tips_02_300ul') # for TIP_CAR_480_A00
# tip_car[4] = STF_L(name='tips_03_300ul') # for TIP_CAR_480_A00

tip_car[0] = TIP_50ul_w_filter_L(name='tips_01_50ul') # for TIP_CAR_NTR_A00
tip_car[1] = TIP_50ul_w_filter_L(name='tips_02_50ul') # for TIP_CAR_NTR_A00


# plt_car[3] = corning_96_wellplate_360ul_flat(name='plt01')
# plt_car[4] = corning_96_wellplate_360ul_flat(name='plt02')
plt_car1[1] = Cos_96_FL(name='source_1')
plt_car1[2] = Cos_96_FL(name='source_2')
plt_car1[3] = Cos_96_FL(name='source_3')
plt_car1[4] = Cos_96_FL(name='source_4')


plt_car2[4] = Cos_96_FL(name='dist1')

# Place Labware

In [9]:

lh.deck.assign_child_resource(tip_car, rails=7) ### OPS OPS TEST POSITION OF THIS LABWARE!!!
### OPS OPS YOU ARE MISSING ONE MFXCARRIER THAT SHOULD BE PLACED ON POSITION RAIL=7

lh.deck.assign_child_resource(plt_car1, rails=27)
lh.deck.assign_child_resource(plt_car2, rails=33)
lh.deck.assign_child_resource(mfx_carrier, rails=13, reassign=True) ## maybe +1 ### OPS OPS TEST POSITION OF THIS LABWARE!!!
lh.deck.assign_child_resource(reservar_carrier1, rails=19, reassign=True) ## maybe ASSUMOING POSITION
lh.deck.assign_child_resource(reservar_carrier2, rails=20, reassign=True) ## maybe ASSUMOING POSITION
lh.deck.assign_child_resource(reservar_carrier3, rails=21, reassign=True) ## maybe ASSUMOING POSITION
lh.deck.assign_child_resource(cooler_carrier1, rails=43, reassign=True) ## APPROX POSITION## APPROX POSITION!!!
lh.deck.assign_child_resource(cooler_carrier2, rails=49, reassign=True) ## APPROX POSITION  !!!

# Simulation create liquids in plates and fill tips

In [10]:
# plt_car = PLT_CAR_L5AC(name='PLT_CAR_L5AC_1')
# plt_car[3] = corning_96_wellplate_360ul_flat(name='plt01')
# plt_car[4] = corning_96_wellplate_360ul_flat(name='plt02')

# tiprack = lh.get_resource("tip01")
# dist1 = lh.get_resource('plt02')
# source_plate = lh.get_resource('plt01')

# if simulate:
  # # define Tips
  # tiprack.fill()
  # plate_1_liquids = [[(None, 50)]]*96
  # # vis.adjust_wells_liquids(source_plate, liquids=plate_1_liquids)
  # source_plate.set_well_liquids(plate_1_liquids)


if simulate:
  # get resources
  tiprack = lh.get_resource("tips_01_50ul")
  source1 = lh.get_resource('source_1')
  source2 = lh.get_resource('source_2')
  source3 = lh.get_resource('source_3')
  source4 = lh.get_resource('source_4')
  dist1 = lh.get_resource('dist1')

  # define Tips
  # await lh.fill_tip_rack(tiprack)

  # define liquids
  plate_1_liquids = [[(None, 100)]]*96
  # await sb.adjust_wells_liquids(source1, liquids=plate_1_liquids)
  # await sb.adjust_wells_liquids(source2, liquids=plate_1_liquids)
  # await sb.adjust_wells_liquids(source3, liquids=plate_1_liquids)
  # await sb.adjust_wells_liquids(source4, liquids=plate_1_liquids)

  # plate_2_liquids = [[(None, 100)], [(None, 500)]]*(96//2)
  # await sb.adjust_wells_liquids(plate_2, liquids=plate_2_liquids)

  source1.set_well_liquids(plate_1_liquids)
  source2.set_well_liquids(plate_1_liquids)
  source3.set_well_liquids(plate_1_liquids)
  source4.set_well_liquids(plate_1_liquids)
  # plate_2.set_well_liquids(plate_2_liquids)

# Only for testing positions on deck 

In [11]:
if not simulate:
  tiprack = lh.get_resource("tips_01_50ul")
  source1 = lh.get_resource('source_1')
  source2 = lh.get_resource('source_2')
  source3 = lh.get_resource('source_3')
  source4 = lh.get_resource('source_4')
  dist1 = lh.get_resource('dist1')

# await lh.pick_up_tips(tiprack['C1'])
# await lh.pick_up_tips(tiprack["C1:E1"])


# await lh.aspirate(source_plate['H1'], vols=[50.0])
# await lh.aspirate(source1[source_well], vols=volume)



In [12]:
# len(channels)

In [26]:

offset = Coordinate(0, 0, 70) # this works for 50ul tips on TIP_CAR_NTR_A00
await lh.pick_up_tips(tiprack["A1"], use_channels=[0], offsets=offset)
# await lh.pick_up_tips(tiprack["A1"], use_channels=[0])

In [27]:

await lh.drop_tips(tiprack["A1"], use_channels=[0], offsets=offset)
# await lh.drop_tips(tiprack["A1:C1"])
# await lh.drop_tips(tip_spots=[trash]*3)
# await lh.return_tips()

In [35]:
await lh.pick_up_tips(tiprack["A2:C2"], offsets=offset)

In [37]:
await lh.aspirate(source1["A1:C1"], vols=[20.0, 20.0, 10.0]) # , swap_speed=100, homogenization_speed=1000

STARFirmwareError: STARFirmwareError(errors={'Pipetting channel 1': AspirationError('Liquid not correctly aspirated'), 'Pipetting channel 3': AspirationError('Liquid not correctly aspirated')}, raw_response=C0ASid0033er99/00 P117/80 P317/80)

In [38]:
await lh.dispense(dist1["A1:C1"], vols=[20.0, 50.0, 10.0]) # swap_speed=100, homogenization_speed=1000, mix_speed=1000

In [39]:
await lh.drop_tips(tiprack["A2:C2"], offsets=offset)

In [34]:
# source_plate = source1
# await lh.aspirate(source1["A1:C1"], vols=[20.0, 40.0, 10.0], swap_speed=100, homogenization_speed=1000)

# await lh.dispense(dist1["A1:C1"], vols=[20.0, 50.0, 10.0], swap_speed=100, homogenization_speed=1000, mix_speed=1000)

# # await lh.drop_tips(tip_spots=[trash]*1, offsets=offsets)
# await lh.drop_tips(tiprack["C1"])


# await lh.drop_tips(tiprack["E1"], use_channels=[2])

# Liquid Commands

In [ ]:
# ## Generate all well positions to pick tips from
# rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
# columns = range(1, 13)
# tip_slot = [f"{row}{col}" for row in rows for col in columns]
# print(tip_slot)

# well_names_sorted = sorted(tip_slot, key=lambda x: (x[1:], x[0]))
# print(well_names_sorted)

In [26]:
tip_slot = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']

In [15]:
t_list = [['10', 'A1', '9', 'A1', '100', 'FSH0345O'], ['10', 'B1', '9', 'B1', '100', 'FSH0345O'], ['10', 'C1', '9', 'C1', '100', 'FSH0345O'], ['10', 'D1', '9', 'D1', '100', 'FSH0345O'], ['10', 'E1', '9', 'E1', '100', 'FSH0345O'], ['10', 'F1', '9', 'F1', '100', 'FSH0345O'], ['10', 'G1', '9', 'G1', '100', 'FSH0345O'], ['10', 'H1', '9', 'H1', '100', 'FSH0345O'], ['10', 'A2', '9', 'A2', '100', 'FSH0345O'], ['10', 'B2', '9', 'B2', '100', 'FSH0345O'], ['10', 'C2', '9', 'C2', '100', 'FSH0345O'], ['10', 'D2', '9', 'D2', '100', 'FSH0345O'], ['10', 'E2', '9', 'E2', '100', 'FSH0345O'], ['10', 'F2', '9', 'F2', '100', 'FSH0345O'], ['10', 'G2', '9', 'G2', '100', 'FSH0345O'], ['10', 'H2', '9', 'H2', '100', 'FSH0345O'], ['10', 'A3', '9', 'A3', '100', 'FSH0345O'], ['10', 'B3', '9', 'B3', '100', 'FSH0345O'], ['10', 'C3', '9', 'C3', '100', 'FSH0345O'], ['10', 'D3', '9', 'D3', '100', 'FSH0345O'], ['10', 'E3', '9', 'E3', '100', 'FSH0345O'], ['10', 'F3', '9', 'F3', '100', 'FSH0345O'], ['10', 'G3', '9', 'G3', '100', 'FSH0345O'], ['10', 'H3', '9', 'H3', '100', 'FSH0345O'], ['10', 'A4', '9', 'A4', '100', 'FSH0345O'], ['10', 'B4', '9', 'B4', '100', 'FSH0345O'], ['10', 'C4', '9', 'C4', '100', 'FSH0345O'], ['10', 'D4', '9', 'D4', '100', 'FSH0345O'], ['10', 'E4', '9', 'E4', '100', 'FSH0345O'], ['10', 'F4', '9', 'F4', '100', 'FSH0345O'], ['10', 'G4', '9', 'G4', '100', 'FSH0345O'], ['10', 'H4', '9', 'H4', '100', 'FSH0345O'], ['10', 'A5', '9', 'A5', '100', 'FSH0345O'], ['10', 'B5', '9', 'B5', '100', 'FSH0345O'], ['10', 'C5', '9', 'C5', '100', 'FSH0345O'], ['10', 'D5', '9', 'D5', '100', 'FSH0345O'], ['10', 'E5', '9', 'E5', '100', 'FSH0345O'], ['10', 'F5', '9', 'F5', '100', 'FSH0345O'], ['7', 'A1', '9', 'G5', '100', 'FSH034AP'], ['7', 'B1', '9', 'H5', '100', 'FSH034AP'], ['7', 'C1', '9', 'A6', '100', 'FSH034AP'], ['7', 'E1', '9', 'B6', '100', 'FSH034AP'], ['7', 'A2', '9', 'C6', '100', 'FSH034AP'], ['7', 'D2', '9', 'D6', '100', 'FSH034AP'], ['7', 'E2', '9', 'E6', '100', 'FSH034AP'], ['7', 'F2', '9', 'F6', '100', 'FSH034AP'], ['7', 'G2', '9', 'G6', '100', 'FSH034AP'], ['7', 'B3', '9', 'H6', '100', 'FSH034AP'], ['7', 'E3', '9', 'A7', '100', 'FSH034AP'], ['7', 'F3', '9', 'B7', '100', 'FSH034AP'], ['7', 'G3', '9', 'C7', '100', 'FSH034AP'], ['7', 'A4', '9', 'D7', '100', 'FSH034AP'], ['7', 'C4', '9', 'E7', '100', 'FSH034AP'], ['7', 'F4', '9', 'F7', '100', 'FSH034AP'], ['7', 'H4', '9', 'G7', '100', 'FSH034AP'], ['7', 'A5', '9', 'H7', '100', 'FSH034AP'], ['7', 'B5', '9', 'A8', '100', 'FSH034AP'], ['7', 'C5', '9', 'B8', '100', 'FSH034AP'], ['7', 'D5', '9', 'C8', '100', 'FSH034AP'], ['7', 'E5', '9', 'D8', '100', 'FSH034AP'], ['7', 'F5', '9', 'E8', '100', 'FSH034AP'], ['7', 'G5', '9', 'F8', '100', 'FSH034AP'], ['7', 'H5', '9', 'G8', '100', 'FSH034AP'], ['7', 'E6', '9', 'H8', '100', 'FSH034AP'], ['7', 'F6', '9', 'A9', '100', 'FSH034AP'], ['7', 'G6', '9', 'B9', '100', 'FSH034AP'], ['7', 'H6', '9', 'C9', '100', 'FSH034AP'], ['7', 'A7', '9', 'D9', '100', 'FSH034AP'], ['7', 'B7', '9', 'E9', '100', 'FSH034AP'], ['7', 'C7', '9', 'F9', '100', 'FSH034AP'], ['7', 'D7', '9', 'G9', '100', 'FSH034AP'], ['7', 'E7', '9', 'H9', '100', 'FSH034AP'], ['7', 'F7', '9', 'A10', '100', 'FSH034AP'], ['7', 'G7', '9', 'B10', '100', 'FSH034AP'], ['7', 'H7', '9', 'C10', '100', 'FSH034AP'], ['7', 'A8', '9', 'D10', '100', 'FSH034AP'], ['7', 'B8', '9', 'E10', '100', 'FSH034AP'], ['7', 'C8', '9', 'F10', '100', 'FSH034AP'], ['7', 'D8', '9', 'G10', '100', 'FSH034AP'], ['7', 'E8', '9', 'H10', '100', 'FSH034AP'], ['7', 'F8', '9', 'A11', '100', 'FSH034AP'], ['7', 'G8', '9', 'B11', '100', 'FSH034AP'], ['7', 'H8', '9', 'C11', '100', 'FSH034AP'], ['7', 'A9', '9', 'D11', '100', 'FSH034AP'], ['7', 'B9', '9', 'E11', '100', 'FSH034AP'], ['7', 'C9', '9', 'F11', '100', 'FSH034AP'], ['7', 'F9', '9', 'G11', '100', 'FSH034AP'], ['7', 'G9', '9', 'H11', '100', 'FSH034AP'], ['7', 'A10', '9', 'A12', '100', 'FSH034AP'], ['7', 'B10', '9', 'B12', '100', 'FSH034AP'], ['7', 'C10', '9', 'C12', '100', 'FSH034AP'], ['7', 'D10', '9', 'D12', '100', 'FSH034AP']]
control_input = {'source_plate': 'biorad_96_wellplate_200ul_pcr', 'dest_plate': 'usascientific_96_wellplate_2.4ml_deep', 'right_pipette': 'p300_single', 'left_pipette': 'p10_single'}

In [16]:
input_slots = {10: source1, 7: source2, 4: source3, 1: source4, 9: dist1}
def find_plate_name(input_slots, source_slot):
    for key, value in input_slots.items():
        if key == source_slot:
            return value
    return None  # Return None if no value found for the given key
test_input = 10
find_plate_name(input_slots, test_input)


Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))

In [17]:
## Testing with

count = 1

source_slot = t_list[count][0]
source_well = t_list[count][1]
dist_slot = t_list[count][2]
dist_well = t_list[count][3]
volume = float(t_list[0][4])
source_plate = find_plate_name(input_slots, int(source_slot))
print('source_plate', source_plate)
print('source_well', source_well)
print(t_list[count])




source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'B1', '9', 'B1', '100', 'FSH0345O']


In [18]:
len(t_list)

92

In [19]:
volumes = []
source_slots = []
source_wells = []
dist_slots = []
dist_wells = []
source_plates = []
dist_plates = []

for count in range(len(t_list)):
  source_slots.append(t_list[count][0])
  source_wells.append(t_list[count][1])
  dist_slots.append(t_list[count][2])
  dist_wells.append(t_list[count][3])
  volumes.append(float(t_list[0][4]))
  source_plates.append(find_plate_name(input_slots, int(source_slots[count])))
  dist_plates.append(find_plate_name(input_slots, int(dist_slots[count])))
  print('source_plate', source_plate)
  print('source_well', source_well)
  print(t_list[count])
  print()


source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'A1', '9', 'A1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'B1', '9', 'B1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'C1', '9', 'C1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'D1', '9', 'D1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'E1', '9', 'E1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'F1',

In [20]:
# # source_plates
# print(dist_slots)
# print(source_slots)
dist_plates

[Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(name=dist1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)),
 Plate(nam

In [21]:

def all_elements_same(source_plates):
    ## test if a list of len=8 contains the same plate if yes aspirate all 8 at the same time if not then aspirate one at a time.
    # Check if the list is empty
    if not source_plates:
        return True
    # Compare all elements to the first element
    first_element = source_plates[0]
    return all(element == first_element for element in source_plates[1:])

In [125]:

## create a tip pick up nested list for doing all work in columns(ei use 8 channels)
multiples_of_8  = [0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96]
tip_pick_up_eight = []
for i in range(len(multiples_of_8)-1):
  if i == 0:
    start = multiples_of_8[i]
  else:
    start = multiples_of_8[i]

  end = multiples_of_8[i+1]
  tip_pick_up_eight.append([start, end])

print(tip_pick_up_eight)

[[0, 8], [8, 16], [16, 24], [24, 32], [32, 40], [40, 48], [48, 56], [56, 64], [64, 72], [72, 80], [80, 88], [88, 96]]


In [126]:
count = 4
start, end = tip_pick_up_eight[count][0] , tip_pick_up_eight[count][1]
print(start, end)

32 40


In [127]:
print(volumes[start:end])
print(source_slots[start:end])
print(source_wells[start:end])
print(dist_slots[start:end])
print(dist_wells[start:end])
print(source_plates[start:end])

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
['10', '10', '10', '10', '10', '10', '7', '7']
['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'A1', 'B1']
['9', '9', '9', '9', '9', '9', '9', '9']
['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5']
[Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_2, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_2, size_x=127.0, size_y=86.0, s

In [128]:
print(start - end)
print(tip_slot[start-1:end])
await lh.pick_up_tips(tiprack[tip_slot[start:end]])

-8
['H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5']
Picking up tips [Pickup(resource=TipSpot(name=tips_01_tipspot_4_0, location=(043.200, 068.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Pickup(resource=TipSpot(name=tips_01_tipspot_4_1, location=(043.200, 059.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Pickup(resource=TipSpot(name=tips_01_tipspot_4_2, location=(043.200, 050.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Pickup(resource=TipSpot(name=tips_01_tipspo

Aspirating [Aspiration(resource=Well(name=source_2_well_0_1, location=(010.570, 062.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)])].


In [131]:

if all_elements_same(source_plates[start:end]):
  source_plate = source_plates[0]
  await lh.aspirate(source_plate[source_wells[start:end]], vols=volumes[start:end], swap_speed=100, homogenization_speed=1000)
else:
  # aspirate one at a time
  for i in range(start, end):
    channel = i - start
    source_plate = source_plates[i]
    await lh.aspirate(source_plate[source_wells[i]], vols=volumes[i], use_channels=[channel])

Aspirating [Aspiration(resource=Well(name=source_1_well_4_0, location=(046.570, 071.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)])].
Aspirating [Aspiration(resource=Well(name=source_1_well_4_1, location=(046.570, 062.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)])].
Aspirating [Aspiration(resource=Well(name=source_1_well_4_2, location=(046.570, 053.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTi

In [132]:
if all_elements_same(dist_plates[start:end]):
  dist1 = dist_plates[0]
  await lh.dispense(dist1[dist_wells[start:end]], vols=volumes[start:end], swap_speed=100, homogenization_speed=1000, mix_speed=1000)
else:
  # dispense one at a time
  for i in range(start, end):
    channel = i - start
    dist1 = dist_plates[i]
    await lh.dispense(dist1[dist_wells[i]], vols=volumes[i], use_channels=[channel])



Dispensing [Dispense(resource=Well(name=dist1_well_4_0, location=(046.570, 071.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)]), Dispense(resource=Well(name=dist1_well_4_1, location=(046.570, 062.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)]), Dispense(resource=Well(name=dist1_well_4_2, location=(046.570, 053.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maxi

In [133]:
await lh.drop_tips(tiprack[tip_slot[start:end]])

Dropping tips [Drop(resource=TipSpot(name=tips_01_tipspot_4_0, location=(043.200, 068.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Drop(resource=TipSpot(name=tips_01_tipspot_4_1, location=(043.200, 059.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Drop(resource=TipSpot(name=tips_01_tipspot_4_2, location=(043.200, 050.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Drop(resource=TipSpot(name=tips_01_tipspot_4_3, location=(043.200, 041.300, -40.500), size_x=9.0, size_y=9.0,

In [ ]:
# # await lh.pick_up_tips(tiprack['C1'])
# await lh.pick_up_tips(tiprack["C1:E1"])


# # await lh.aspirate(source_plate['H1'], vols=[50.0])
# # await lh.aspirate(source1[source_well], vols=volume)

# # source_plate = lh.get_resource('plt01')
# await lh.aspirate(source_plate["A1:C1"], vols=[20.0, 40.0, 10.0], swap_speed=100, homogenization_speed=1000)

# await lh.dispense(dist1["A1:C1"], vols=[20.0, 50.0, 10.0], swap_speed=100, homogenization_speed=1000, mix_speed=1000)

# # await lh.drop_tips(tip_spots=[trash]*1, offsets=offsets)
# await lh.drop_tips(tiprack["C1"])


# await lh.drop_tips(tiprack["E1"], use_channels=[2])



Plate(name=source_1, size_x=127.76, size_y=85.47, size_z=14.22, location=(000.000, 271.500, 000.000))

In [ ]:
print(lh.deck.summary())

Rail     Resource                   Type                Coordinates (mm)
(-13) ├── trash_core96               Trash               (-232.100, 110.300, 189.000)
      │
(0)   ├── my_shaker_carrier          ShakerCarrier       (077.500, 063.000, 100.000)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(7)   ├── tip_carrier                TipCarrier          (235.000, 063.000, 100.000)
      │   ├── <empty>
      │   ├── tip01                  TipRack             (252.900, 241.800, 174.450)
      │   ├── tip02                  TipRack             (252.900, 337.800, 174.450)
      │   ├── <empty>
      │   ├── <empty>
      │
(13)  ├── my_mfx_carrier             MFXCarrier          (370.000, 063.000, 100.000)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(19)  ├── my_RGT_CAR_3R_A01_1        TubeCarrier         (505.000, 063.000, 100.000)
      │   ├──

# Shutdown

In [40]:
await lh.stop()

# Alternative tip picking test


# Building Hamilton deck for Hamilton in B221 R032